In [1]:
import pandas as pd
import numpy as np
import zipfile

import pyomo.environ as pyo
import plotly.express as px

NA = 9999

In [2]:
census_path = "input/RP2019_INDCVIZA_csv.zip"
clustering_path = "input/clusters_2019.csv"
cplex_path = "/home/shoerl/.cplex_community/cplex/bin/x86-64_linux/cplex"

output_path = "output"

regularization = 0.0
upper_factor_bound = 1.75
lower_factor_bound = 0.25

### Prepare census data

Same as in other notebook. Eventually, we could externalize this to have just one notebook that cleans the census data.

In [3]:
with zipfile.ZipFile(census_path) as archive:
    with archive.open("FD_INDCVIZA_2019.csv") as f:
        df_census = pd.read_csv(f, sep = ";", usecols = [
            "VOIT", "INPER", "IPONDI", "ILT", "ILETUD", "ETUD", "TACT"
        ], dtype = {
            "ILETUD": str, "ILT": str, "INPER": str, "IPONDI": float, "VOIT": str,
            "ETUD": str, "TACT": str
        })

In [4]:
# Clean weight
df_census["weight"] = df_census["IPONDI"]

In [5]:
# Clean household size
df_census["household_size"] = pd.to_numeric(df_census["INPER"], errors = "coerce").fillna(NA).astype(int)

df_census.loc[df_census["household_size"] > 6, "household_size"] = NA

In [6]:
# Clean number of cars
df_census["number_of_cars"] = pd.to_numeric(df_census["VOIT"], errors = "coerce").fillna(NA).astype(int)

df_census.loc[df_census["number_of_cars"] > 3, "number_of_cars"] = NA

In [7]:
# Clean location type
df_census["studies"] = df_census["ETUD"] == "1"
df_census["employed"] = df_census["TACT"] == "11"

# ... location type of employed persons
df_census.loc[df_census["employed"],
    "location_type"] = df_census.loc[df_census["employed"], "ILT"]

# ... location type of studying persons
df_census.loc[df_census["studies"], 
    "location_type"] = df_census.loc[df_census["studies"], "ILETUD"]

# ... cleanup
df_census.loc[~df_census["location_type"].isin(["1", "2", "3"]), "location_type"] = str(NA)
df_census["location_type"] = df_census["location_type"].astype(int)

### Merge with personas

In [8]:
df_clusters = pd.read_csv(clustering_path)
df_census["persona"] = df_clusters["ID"].values

persona_count = len(df_census["persona"].unique())
assert persona_count == 16
assert set(df_census["persona"].unique()) == set(range(persona_count))

### Prepare optimization data

In [9]:
household_size_values = [1, 2, 3, 4, 5, 6, NA]
household_size_counts = df_census.groupby(["persona", "household_size"])["weight"].sum().reindex(pd.MultiIndex.from_product([
    np.arange(16), household_size_values])).fillna(0.0).values.reshape((persona_count, len(household_size_values)))

number_of_cars_values = [0, 1, 2, 3, NA]
number_of_cars_counts = df_census.groupby(["persona", "number_of_cars"])["weight"].sum().reindex(pd.MultiIndex.from_product([
    np.arange(16), number_of_cars_values])).fillna(0.0).values.reshape((persona_count, len(number_of_cars_values)))

location_type_values = [1, 2, 3, NA]
location_type_counts = df_census.groupby(["persona", "location_type"])["weight"].sum().reindex(pd.MultiIndex.from_product([
    np.arange(16), location_type_values])).fillna(0.0).values.reshape((persona_count, len(location_type_values)))

persona_counts = df_census.groupby("persona")["weight"].sum().reindex(np.arange(16)).fillna(0.0).values.reshape((persona_count,))
total_count = np.sum(persona_counts)

In [10]:
df_household_size_targets = pd.read_parquet(
    "{}/distribution_household_size.parquet".format(output_path))

df_number_of_cars_targets = pd.read_parquet(
    "{}/distribution_number_of_cars.parquet".format(output_path))

df_location_type_targets = pd.read_parquet(
    "{}/distribution_location_type.parquet".format(output_path))

### Optimization

Given a set of attributes $a \in \mathcal{A}$ with their unique values $v^a \in \mathcal{V}_a$, we can define the number of times that attribute $a$ takes value $v$ over a list of personas $p \in \mathcal{P}$ as $n^a_{vp}$. Furthermore, we have a desired marginal distribution of attribute $a$ given as $m^a_v$.

We can then introduce weighting factors per persona as $\omega_p \geq 0$ that we can use to calculate the marginal distribution, given a certain distribution of personas:

$m'^a_v = \sum_p n^a_{vp} \cdot \omega_p$

The question is now how the factors $\omega$ should be chosen such that the resulting marginal distributions fit well to the desired marginal distribution:

$
\text{minimize}_{\omega_p} \sum_p \left( m'^a_v - m^a_v \right)^2
$

subject to the conditions above. Furthermore, we require that the factors stay within a certain bound:

$
\omega_p \leq \overline \omega
$

$
\omega_p \geq \underline \omega
$

In [11]:
scenarios = df_household_size_targets["scenario"].unique()
instances = []

# Create one optimization model per scenario
for scenario in df_household_size_targets["scenario"].unique():
    if scenario == "Baseline": continue
    
    model = pyo.ConcreteModel()

    # Obtain target values per scenario
    household_size_targets = df_household_size_targets[
        df_household_size_targets["scenario"] == scenario].set_index("household_size").loc[household_size_values, "weight"].values

    number_of_cars_targets = df_number_of_cars_targets[
        df_number_of_cars_targets["scenario"] == scenario].set_index("number_of_cars").loc[number_of_cars_values, "weight"].values

    location_type_targets = df_location_type_targets[
        df_location_type_targets["scenario"] == scenario].set_index("location_type").loc[location_type_values, "weight"].values

    # Define person shares
    model.x = pyo.Var(list(range(persona_count)), 
        domain = pyo.NonNegativeReals, 
        initialize = np.ones((persona_count,)))

    def upper_constraint(model, p):
        return model.x[p] <= upper_factor_bound

    model.upper_constraint = pyo.Constraint(range(persona_count), rule = upper_constraint)

    def lower_constraint(model, p):
        return model.x[p] >= lower_factor_bound

    model.lower_constraint = pyo.Constraint(range(persona_count), rule = lower_constraint)

    def total_constraint(model):
        return sum([persona_counts[p] * model.x[p] for p in range(persona_count)]) == np.sum(persona_counts)

    model.total_constraint = pyo.Constraint(rule = total_constraint)

    # Define objective
    def objective_function(model):
        objective = 0.0

        objective += sum([
            (sum([
                household_size_counts[p, k] * model.x[p]
                for p in range(persona_count)
            ]) - household_size_targets[k])**2
            for k in range(len(household_size_values))
        ]) / np.max(household_size_counts)

        objective += sum([
            (sum([
                number_of_cars_counts[p, k] * model.x[p]
                for p in range(persona_count)
            ]) - number_of_cars_targets[k])**2
            for k in range(len(number_of_cars_values))
        ]) / np.max(number_of_cars_counts)

        objective += sum([
            (sum([
                location_type_counts[p, k] * model.x[p]
                for p in range(persona_count)
            ]) - location_type_targets[k])**2
            for k in range(len(location_type_values))
        ]) / np.max(location_type_counts)

        objective += regularization * total_count * sum([
            (model.x[p] - 1)**2
            for p in range(persona_count)
        ])

        return objective

    model.objective = pyo.Objective(expr = objective_function, sense = pyo.minimize)

    instances.append({ 
        "scenario": scenario, "model": model,
        "household_size_targets": household_size_targets,
        "number_of_cars_targets": number_of_cars_targets,
        "location_type_targets": location_type_targets
    })

In [12]:
# Solve models using CPLEX
df_output = []

for instance in instances:
    model = instance["model"]

    optimizer = pyo.SolverFactory("cplex", executable = cplex_path)
    solution = optimizer.solve(model)

    assert solution.solver.status == "ok"
    assert solution.solver.termination_condition == "optimal"

    factors = np.array([
        model.x[p].value for p in range(persona_count)
    ])

    instance["factors"] = factors

    df_output.append(pd.DataFrame({
        "persona": np.arange(persona_count),
        "weight": persona_counts * factors,
        "scenario": instance["scenario"]
    }))

df_output.append(pd.DataFrame({
    "persona": np.arange(persona_count),
    "weight": persona_counts,
    "scenario": "Baseline"
}))

df_output = pd.concat(df_output)

In [13]:
px.bar(
    df_output.sort_values(by = "scenario"), x = "persona", y = "weight", color = "scenario", barmode = "group",
    title = "Persona distribution by scenario")

In [14]:
# Compare household size distribution
for instance in instances:
    df_comparison = pd.DataFrame({
        "household_size": household_size_values,
        "initial_share": np.sum(household_size_counts, axis = 0),
        "target_share": instance["household_size_targets"],
        "scenario_share": np.sum(household_size_counts * instance["factors"].reshape((-1, 1)), axis = 0),
    })

    df_plot = df_comparison.melt("household_size", ["initial_share", "target_share", "scenario_share"], value_name = "share")
    df_plot["case"] = df_plot["variable"].replace("_share", "")
    df_plot["household_size"] = df_plot["household_size"].astype(str).replace({ str(NA): "NA" })
    
    figure = px.bar(df_plot, x = "household_size", y = "share", color = "case", barmode = "group",
        title = "Household size distribution (Scenario {})".format(instance["scenario"]))
    
    figure.show()

In [15]:
# Compare number of cars distribution
for instance in instances:
    df_comparison = pd.DataFrame({
        "number_of_cars": number_of_cars_values,
        "initial_share": np.sum(number_of_cars_counts, axis = 0),
        "target_share": instance["number_of_cars_targets"],
        "scenario_share": np.sum(number_of_cars_counts * instance["factors"].reshape((-1, 1)), axis = 0),
    })

    df_plot = df_comparison.melt("number_of_cars", ["initial_share", "target_share", "scenario_share"], value_name = "share")
    df_plot["case"] = df_plot["variable"].replace("_share", "")
    df_plot["number_of_cars"] = df_plot["number_of_cars"].astype(str).replace({ str(NA): "NA" })
    
    figure = px.bar(df_plot, x = "number_of_cars", y = "share", color = "case", barmode = "group",
        title = "Number of cars distribution (Scenario {})".format(instance["scenario"]))
    
    figure.show()

In [16]:
# Compare locatyion type distribution
for instance in instances:
    df_comparison = pd.DataFrame({
        "location_type": location_type_values,
        "initial_share": np.sum(location_type_counts, axis = 0),
        "target_share": instance["location_type_targets"],
        "scenario_share": np.sum(location_type_counts * instance["factors"].reshape((-1, 1)), axis = 0),
    })

    df_plot = df_comparison.melt("location_type", ["initial_share", "target_share", "scenario_share"], value_name = "share")
    df_plot["case"] = df_plot["variable"].replace("_share", "")
    df_plot["location_type"] = df_plot["location_type"].astype(str).replace({ str(NA): "NA" })
    
    figure = px.bar(df_plot, x = "location_type", y = "share", color = "case", barmode = "group",
        title = "Location type distribution (Scenario {})".format(instance["scenario"]))
    
    figure.show()

### Output

In [17]:
df_output[[
    "persona", "scenario", "weight"
]].to_parquet("{}/distribution_persona.parquet".format(output_path))

In [18]:
df_output[[
    "persona", "scenario", "weight"
]]

,persona,scenario,weight
0,0,S2,1.747566e+06
1,1,S2,6.444983e+05
2,2,S2,1.906705e+06
3,3,S2,5.663389e+05
4,4,S2,1.250667e+06
...,...,...,...
11,11,Baseline,7.742695e+05
12,12,Baseline,3.797709e+05
13,13,Baseline,6.594512e+05
14,14,Baseline,4.179516e+05
